- Original ALLO features from DoGSiteScorer cannot be used.
- Highest probability pocket for each PDB is taken as the positive prediction.

<br>

In [1]:
pdbs = ['8sgj', 'AF-A0A1D8PQM9-F1', '7l6r', '6yhr', '7xlq', '5b0u', '8vw5', '5uak', '4jqi']

# DoGSiteScorer

In [2]:
import os, requests, json, time

In [3]:
def get_state(location):
    response = requests.get(location)
    return response.status_code, response

In [4]:
def upload_pdb(file_path, max_retries=5, wait_time=5):
    pdb_upload_url = "https://proteins.plus/api/pdb_files_rest"
    headers = {"Accept": "application/json"}
    
    pdb_file = open(file_path, "rb")
    files = {"pdb_file[pathvar]": pdb_file}
    upload_response = requests.post(pdb_upload_url, headers=headers, files=files)
    pdb_file.close()
    
    location = upload_response.json().get("location")

    time.sleep(10)
    status_code, response = get_state(location)

    retries = 0
    while status_code != 200 and retries < max_retries:
        wait = wait_time*(retries+1)
        print("({}) {}: try again after {} seconds".format(file_path, response.json().get("message"), wait))
        time.sleep(wait)
        status_code, response = get_state(location)
        retries += 1

    return response.json().get("id")

In [5]:
def run_dogsite_analysis(pdb_id, max_retries=5, wait_time=30):
    dogsite_url = "https://proteins.plus/api/dogsite_rest"
    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json"
    }
    data = {
        "dogsite": {
            "pdbCode": pdb_id,
            "analysisDetail": "0",
            "bindingSitePredictionGranularity": "1",
            "ligand": "",
            "chain": ""
        }
    }

    upload_response = requests.post(dogsite_url, headers=headers, data=json.dumps(data))
    location = upload_response.json().get("location")
    
    time.sleep(10)
    status_code, response = get_state(location)

    retries = 0
    while status_code != 200 and retries < max_retries:
        wait = wait_time*(retries+1)
        print("({}) {}: try again after {} seconds".format(pdb_id, response.json().get("message"), wait))
        time.sleep(wait)
        status_code, response = get_state(location)
        retries += 1

    return response.json()

In [6]:
def download(url, path):
    response = requests.get(url)
    if response.status_code == 200:
        with open("{}/{}".format(path, url.split("/")[-1]), "wb") as f:
            f.write(response.content)

In [7]:
for pdb in pdbs:
    pdb_path = "../structures/{}.pdb".format(pdb)
    if not os.path.isdir(pdb):
        os.makedirs(pdb)
        pockets = "{}/pockets".format(pdb)
        if not os.path.isdir(pockets):
            os.makedirs(pockets)

        pdb_id = upload_pdb(pdb_path)
        
        result = run_dogsite_analysis(pdb_id)
        
        if result is not None:
            for url in result.get("pockets", []) + result.get("residues", []):
                download(url, pockets)
            
            download(result.get("descriptor_explanation"), pdb)
            download(result.get("result_table"), pdb)

(8vw5pdb9c7b998f-973d-4970-9734-43e8c162cd68) Job exists and is still in 'processing' state: try again after 30 seconds


## Old vs. new version

In [8]:
import pandas as pd

In [9]:
old = pd.read_csv("allo/src/test_input/AS091022202_3PJG_complex.txt", sep="\t")
old

,name,lig_cov,poc_cov,lig_name,4A_crit,volume,hull,surface,lid,depth,...,I,N,DA,DC,DG,DT,DN,UNK,simpleScore,drugScore
0,AS091022202_3PJG_complex_chains_desc_P_1,0.0000,0.00000,non,0,926.336,1125.44,956.64,168.80,18.40000,...,0,0,0,0,0,0,0,0,0.537801,0.520085
1,AS091022202_3PJG_complex_chains_desc_P_2,0.0000,0.00000,non,0,782.464,917.44,707.52,209.92,29.12320,...,0,0,0,0,0,0,0,0,0.445098,0.417353
2,AS091022202_3PJG_complex_chains_desc_P_3,23.7288,7.66129,UGA_A_90,1,555.520,659.52,551.68,107.84,16.04490,...,0,0,0,0,0,0,0,0,0.404041,0.363490
3,AS091022202_3PJG_complex_chains_desc_P_4,0.0000,0.00000,non,0,285.312,419.20,406.24,12.96,13.93700,...,0,0,0,0,0,0,0,0,0.282709,0.100086
4,AS091022202_3PJG_complex_chains_desc_P_5,0.0000,0.00000,non,0,251.328,356.96,351.20,5.76,12.65540,...,0,0,0,0,0,0,0,0,0.365878,0.202703
5,AS091022202_3PJG_complex_chains_desc_P_6,0.0000,0.00000,non,0,148.544,198.24,152.32,45.92,7.16659,...,0,0,0,0,0,0,0,0,0.256271,0.420902
6,AS091022202_3PJG_complex_chains_desc_P_7,42.3729,85.42170,UGA_A_90,1,131.264,178.08,143.20,34.88,9.55824,...,0,0,0,0,0,0,0,0,0.532231,0.652536
7,AS091022202_3PJG_complex_chains_desc_P_8,0.0000,0.00000,non,0,116.224,169.76,148.96,20.80,8.88144,...,0,0,0,0,0,0,0,0,0.053244,0.447991
8,AS091022202_3PJG_complex_chains_desc_P_9,0.0000,0.00000,non,0,115.776,168.00,120.48,47.52,8.40952,...,0,0,0,0,0,0,0,0,0.000000,0.266754
9,AS091022202_3PJG_complex_chains_desc_P_10,0.0000,0.00000,non,0,113.344,173.44,152.64,20.80,9.67471,...,0,0,0,0,0,0,0,0,0.231786,0.324773


In [18]:
new = pd.read_csv("8vw5/8vw5pdb9c7b998f-973d-4970-9734-43e8c162cd68_desc.txt", sep="\t")
new

,name,lig_cov,poc_cov,lig_name,volume,enclosure,surface,depth,surf/vol,lid/hull,...,MET,PHE,PRO,SER,THR,TRP,TYR,VAL,simpleScore,drugScore
0,P_0,0.0,0.0,NaN,1477.89,0.14,1833.30,25.28,1.240485,-,...,0,4,2,4,5,2,3,1,0.64,0.812764
1,P_1,0.0,0.0,NaN,300.22,0.12,354.88,14.59,1.182066,-,...,0,2,2,0,0,1,0,0,0.16,0.630137
2,P_2,0.0,0.0,NaN,276.67,0.08,395.50,13.61,1.429501,-,...,1,0,0,2,1,0,3,0,0.05,0.537525
3,P_3,0.0,0.0,NaN,235.20,0.29,362.44,8.75,1.540986,-,...,0,0,2,2,1,0,1,0,0.04,0.343150
4,P_4,0.0,0.0,NaN,186.43,0.24,308.57,8.25,1.655152,-,...,0,1,1,0,1,0,2,0,0.08,0.281039
5,P_5,0.0,0.0,NaN,167.62,0.11,286.98,10.03,1.712087,-,...,0,2,1,1,0,0,2,0,0.00,0.352875
6,P_6,0.0,0.0,NaN,122.37,0.18,203.57,8.22,1.663561,-,...,0,1,0,0,1,0,1,0,0.00,0.208202
7,P_7,0.0,0.0,NaN,109.76,0.11,238.14,7.35,2.169643,-,...,1,0,1,0,0,1,1,1,0.01,0.226149


### Different columns

In [12]:
set(new.columns) - set(old.columns)

{'enclosure', 'hydrophobic_interactions'}

In [13]:
new[list(set(new.columns) - set(old.columns))]

,hydrophobic_interactions,enclosure
0,86,0.14
1,21,0.12
2,12,0.08
3,13,0.29
4,25,0.24
5,9,0.11
6,9,0.18
7,16,0.11


In [17]:
set(old.columns) - set(new.columns)

{'4A_crit',
 'A',
 'C',
 'CSO',
 'DA',
 'DC',
 'DG',
 'DN',
 'DT',
 'G',
 'I',
 'N',
 'U',
 'UNK',
 'aromat',
 'hull',
 'lid',
 'sumAA'}

In [15]:
old[sorted(set(old.columns) - set(new.columns), key=lambda x: old.columns.tolist().index(x))]

,4A_crit,hull,lid,aromat,sumAA,CSO,A,C,G,U,I,N,DA,DC,DG,DT,DN,UNK
0,0,1125.44,168.80,17,40,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,917.44,209.92,9,36,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,659.52,107.84,9,27,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,419.20,12.96,4,22,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,356.96,5.76,15,19,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,198.24,45.92,14,8,0,0,0,0,0,0,0,0,0,0,0,0,0
6,1,178.08,34.88,10,6,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,169.76,20.80,3,10,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,168.00,47.52,0,10,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,173.44,20.80,4,10,0,0,0,0,0,0,0,0,0,0,0,0,0


**4A_crit is part of the labelling system of ALLO and must be kept/added.**

<br>

### Missing data in new output

In [20]:
new.dtypes.sort_values(ascending=False)

name                         object
ellVol                       object
lid/hull                     object
apolarAA                    float64
posAA                       float64
simpleScore                 float64
hydrophobicity              float64
ell b/a                     float64
ell c/a                     float64
surf/vol                    float64
depth                       float64
surface                     float64
enclosure                   float64
volume                      float64
lig_name                    float64
poc_cov                     float64
lig_cov                     float64
polarAA                     float64
drugScore                   float64
negAA                       float64
accept                        int64
VAL                           int64
TYR                           int64
TRP                           int64
THR                           int64
SER                           int64
PRO                           int64
PHE                         

In [21]:
new[["lid/hull", "ellVol"]]

,lid/hull,ellVol
0,-,-
1,-,-
2,-,-
3,-,-
4,-,-
5,-,-
6,-,-
7,-,-


### Patch

- Create a fake new aplc.tsv to train the neural network using only the features that are available.
- Process all outputs to remove features not available in old output

#### Training data

In [21]:
if not os.path.isfile("allo/data/aplc_original.tsv"):
    os.system("mv allo/data/aplc.tsv allo/data/aplc_original.tsv")

0

In [22]:
aplc = pd.read_csv("allo/data/aplc_original.tsv", sep="\t")
aplc

,name,lig_cov,poc_cov,lig_name,4A_crit,volume,hull,surface,lid,depth,...,I,N,DA,DC,DG,DT,DN,UNK,simpleScore,drugScore
0,AS001000501_3UO9_complex_chains_desc_P_1,0.00000,0.000000,non,0,661.207,749.567,678.2210,71.346400,24.90670,...,0,0,0,0,0,0,0,0,0.468934,0.271149
1,AS001000501_3UO9_complex_chains_desc_P_2,0.00000,0.000000,non,0,492.799,558.164,434.6690,123.495000,17.28740,...,0,0,0,0,0,0,0,0,0.262105,0.215701
2,AS001000501_3UO9_complex_chains_desc_P_3,88.13560,72.303800,04A_B_2,1,480.683,558.164,502.8630,55.300700,20.22080,...,0,0,0,0,0,0,0,0,0.593739,0.695285
3,AS001000501_3UO9_complex_chains_desc_P_4,0.00000,0.000000,non,0,463.811,565.327,453.2930,112.034000,20.41120,...,0,0,0,0,0,0,0,0,0.285292,0.254757
4,AS001000501_3UO9_complex_chains_desc_P_5,0.00000,0.000000,non,0,398.013,519.196,388.2510,130.945000,20.24210,...,0,0,0,0,0,0,0,0,0.260897,0.147678
5,AS001000501_3UO9_complex_chains_desc_P_6,0.00000,0.000000,non,0,381.601,493.694,428.6520,65.042700,21.60460,...,0,0,0,0,0,0,0,0,0.386060,0.136666
6,AS001000501_3UO9_complex_chains_desc_P_7,0.00000,0.000000,non,0,373.626,436.102,369.9130,66.188900,17.80990,...,0,0,0,0,0,0,0,0,0.510129,0.285342
7,AS001000501_3UO9_complex_chains_desc_P_8,0.00000,0.000000,non,0,373.166,496.273,412.3190,83.953800,22.42460,...,0,0,0,0,0,0,0,0,0.391934,0.178084
8,AS001000501_3UO9_complex_chains_desc_P_9,0.00000,0.000000,non,0,353.380,476.789,396.8470,79.942400,24.36580,...,0,0,0,0,0,0,0,0,0.397897,0.088986
9,AS001000501_3UO9_complex_chains_desc_P_10,3.38983,0.591447,04A_B_2,0,337.122,408.594,303.1510,105.444000,17.75350,...,0,0,0,0,0,0,0,0,0.144927,0.317734


In [23]:
assert set(aplc.columns) == set(old.columns)

In [24]:
columns = [c for c in aplc.columns if c not in set(
    list( set(new.columns) - set(aplc.columns) )
    + list( set(aplc.columns) - set(new.columns) )
    + ["lid/hull", "ellVol"]
) - {"4A_crit",}]
columns

['name',
 'lig_cov',
 'poc_cov',
 'lig_name',
 '4A_crit',
 'volume',
 'surface',
 'depth',
 'surf/vol',
 'ell c/a',
 'ell b/a',
 'siteAtms',
 'accept',
 'donor',
 'hydrophobicity',
 'metal',
 'Cs',
 'Ns',
 'Os',
 'Ss',
 'Xs',
 'negAA',
 'posAA',
 'polarAA',
 'apolarAA',
 'ALA',
 'ARG',
 'ASN',
 'ASP',
 'CYS',
 'GLN',
 'GLU',
 'GLY',
 'HIS',
 'ILE',
 'LEU',
 'LYS',
 'MET',
 'PHE',
 'PRO',
 'SER',
 'THR',
 'TRP',
 'TYR',
 'VAL',
 'simpleScore',
 'drugScore']

In [25]:
fake_aplc = aplc[columns]
fake_aplc

,name,lig_cov,poc_cov,lig_name,4A_crit,volume,surface,depth,surf/vol,ell c/a,...,MET,PHE,PRO,SER,THR,TRP,TYR,VAL,simpleScore,drugScore
0,AS001000501_3UO9_complex_chains_desc_P_1,0.00000,0.000000,non,0,661.207,678.2210,24.90670,0.549061,0.120963,...,2,3,2,5,0,0,2,1,0.468934,0.271149
1,AS001000501_3UO9_complex_chains_desc_P_2,0.00000,0.000000,non,0,492.799,434.6690,17.28740,0.472144,0.117456,...,2,1,1,4,0,0,1,1,0.262105,0.215701
2,AS001000501_3UO9_complex_chains_desc_P_3,88.13560,72.303800,04A_B_2,1,480.683,502.8630,20.22080,0.559987,0.081842,...,0,2,0,0,0,0,2,0,0.593739,0.695285
3,AS001000501_3UO9_complex_chains_desc_P_4,0.00000,0.000000,non,0,463.811,453.2930,20.41120,0.523148,0.093727,...,2,2,2,3,0,0,0,2,0.285292,0.254757
4,AS001000501_3UO9_complex_chains_desc_P_5,0.00000,0.000000,non,0,398.013,388.2510,20.24210,0.522158,0.090831,...,0,1,3,1,0,0,0,3,0.260897,0.147678
5,AS001000501_3UO9_complex_chains_desc_P_6,0.00000,0.000000,non,0,381.601,428.6520,21.60460,0.601286,0.055553,...,2,0,0,1,3,0,0,2,0.386060,0.136666
6,AS001000501_3UO9_complex_chains_desc_P_7,0.00000,0.000000,non,0,373.626,369.9130,17.80990,0.529967,0.145510,...,0,2,2,1,0,0,0,0,0.510129,0.285342
7,AS001000501_3UO9_complex_chains_desc_P_8,0.00000,0.000000,non,0,373.166,412.3190,22.42460,0.591451,0.048928,...,0,2,2,1,1,1,1,2,0.391934,0.178084
8,AS001000501_3UO9_complex_chains_desc_P_9,0.00000,0.000000,non,0,353.380,396.8470,24.36580,0.601128,0.032981,...,0,2,3,1,1,1,0,2,0.397897,0.088986
9,AS001000501_3UO9_complex_chains_desc_P_10,3.38983,0.591447,04A_B_2,0,337.122,303.1510,17.75350,0.481347,0.102600,...,0,1,1,1,0,0,0,1,0.144927,0.317734


In [26]:
fake_aplc.to_csv("../data/aplc.tsv", sep="\t", index=False) #"allo/data/aplc_new.tsv"

#### New data

In [27]:
for pdb in pdbs:
    if os.path.isdir(pdb):
        if "AF" not in pdb:
            inf = next((f for f in os.listdir(pdb) if f.startswith(pdb) and f.endswith("_desc.txt")))
        else:
            inf = next((f for f in os.listdir(pdb) if f.startswith(pdb.replace("-", "").lower()) and f.endswith("_desc.txt")))
        (
            pd.read_csv("{}/{}".format(pdb, inf), sep="\t")
            .assign(**{"4A_crit": 0})
            [columns]
            .to_csv("{}/{}.txt".format(pdb, pdb), sep="\t", index=False)
        )

# Predictions

In [28]:
data = pd.concat((
    (
        pd.read_csv("{}/{}.txt".format(pdb, pdb), sep="\t")
        .assign(
            name=lambda x: x["name"].apply(
                lambda y: "{}_{}".format(pdb, y)
            )
        )
        [columns]
    )
    for pdb in pdbs
    if os.path.isdir(pdb) and os.path.isfile("{}/{}.txt".format(pdb, pdb))
))
data

,name,lig_cov,poc_cov,lig_name,4A_crit,volume,surface,depth,surf/vol,ell c/a,...,MET,PHE,PRO,SER,THR,TRP,TYR,VAL,simpleScore,drugScore
0,8sgj_P_0,0.0,0.0,NaN,0,1143.23,1432.09,25.86,1.252670,0.12,...,1,3,2,2,1,0,2,1,0.60,0.804140
1,8sgj_P_1,0.0,0.0,NaN,0,1022.03,1076.22,27.75,1.053022,0.15,...,1,2,0,2,4,1,2,6,0.61,0.814287
2,8sgj_P_2,0.0,0.0,NaN,0,686.75,826.21,17.47,1.203072,0.26,...,2,1,1,4,0,1,0,2,0.45,0.809774
3,8sgj_P_3,0.0,0.0,NaN,0,537.44,657.32,16.60,1.223057,0.10,...,0,1,0,1,1,0,3,2,0.34,0.742773
4,8sgj_P_4,0.0,0.0,NaN,0,517.87,634.08,11.68,1.224400,0.13,...,1,0,1,2,4,0,0,3,0.32,0.622482
5,8sgj_P_5,0.0,0.0,NaN,0,506.63,769.00,19.85,1.517873,0.14,...,0,0,0,2,2,0,1,1,0.28,0.795941
6,8sgj_P_6,0.0,0.0,NaN,0,457.83,565.23,20.85,1.234585,0.09,...,0,1,2,7,2,1,1,6,0.19,0.821576
7,8sgj_P_7,0.0,0.0,NaN,0,339.33,475.99,16.38,1.402735,0.10,...,1,3,2,2,2,0,0,5,0.27,0.706929
8,8sgj_P_8,0.0,0.0,NaN,0,293.00,429.66,16.24,1.466416,0.06,...,1,2,0,1,1,2,0,0,0.27,0.677591
9,8sgj_P_9,0.0,0.0,NaN,0,225.54,288.27,13.17,1.278132,0.21,...,0,0,3,0,1,1,3,0,0.01,0.500000


In [29]:
data.to_csv("data.txt", sep="\t", index=False)

In [30]:
os.system("cd allo/src && python rank_nn.py {}/data.txt".format( os.getcwd() ))

0

# Process

In [31]:
preds = pd.read_csv("data_nn_out.txt")
preds

,name,predicted probability
0,6yhr_P_0,0.10181
1,5b0u_P_0,0.07064
2,AF-A0A1D8PQM9-F1_P_0,0.06321
3,8vw5_P_0,0.04440
4,8sgj_P_1,0.03776
5,7xlq_P_0,0.02777
6,7l6r_P_9,0.02764
7,5uak_P_0,0.02725
8,8sgj_P_0,0.02561
9,AF-A0A1D8PQM9-F1_P_1,0.02547


In [32]:
preds = preds.assign(
    **dict(
        zip(
            ("pdb", "pocket"),
            zip(*preds["name"].apply(lambda x: x.split("_", 1)).tolist())
        )
    )
)

preds

,name,predicted probability,pdb,pocket
0,6yhr_P_0,0.10181,6yhr,P_0
1,5b0u_P_0,0.07064,5b0u,P_0
2,AF-A0A1D8PQM9-F1_P_0,0.06321,AF-A0A1D8PQM9-F1,P_0
3,8vw5_P_0,0.04440,8vw5,P_0
4,8sgj_P_1,0.03776,8sgj,P_1
5,7xlq_P_0,0.02777,7xlq,P_0
6,7l6r_P_9,0.02764,7l6r,P_9
7,5uak_P_0,0.02725,5uak,P_0
8,8sgj_P_0,0.02561,8sgj,P_0
9,AF-A0A1D8PQM9-F1_P_1,0.02547,AF-A0A1D8PQM9-F1,P_1


In [33]:
from Bio import PDB

def process_pocket(pdb, pocket):
    if "AF" not in pdb:
        f = next((
            f 
            for f in os.listdir("{}/pockets".format(pdb)) 
            if f.startswith(pdb) and f.endswith("_{}_res.pdb".format(pocket))
        ))
    else:
        f = next((
            f 
            for f in os.listdir("{}/pockets".format(pdb)) 
            if f.startswith(pdb.replace("-", "").lower()) and f.endswith("_{}_res.pdb".format(pocket))
        ))
    s = PDB.PDBParser(QUIET=True).get_structure("protein", "{}/pockets/{}".format(pdb, f))
    return pd.DataFrame(
        {
            "auth_asym_id": chain.id,
            "auth_seq_id": res.get_id()[1],
            "pdbx_PDB_ins_code": res.get_id()[2] if res.get_id()[2] != " " else "?"
        }
        for model in s
        for chain in model
        for res in chain
    ).drop_duplicates()

In [34]:
results = {}

for pdb, pockets in preds.groupby("pdb"):
    d = {}
    
    pockets = (
        pockets
        .sort_values(["predicted probability", "pocket"], ascending=[False, True])
        [["predicted probability", "pocket"]]
        .values.tolist()
    )

    # Top pocket
    prob, pocket = pockets.pop(0)
    d[pocket] = {
        "prob": prob,
        "pred": 1,
        "residues": process_pocket(pdb, pocket)
    }
    
    for prob, pocket in pockets:
        d[pocket] = {
            "prob": prob,
            "pred": 0,
            "residues": process_pocket(pdb, pocket)
        }
        
    results[pdb] = d

results

{'4jqi': {'P_0': {'pred': 1,
   'prob': 0.023459999999999998,
   'residues':    auth_asym_id  auth_seq_id pdbx_PDB_ins_code
   0             A           26                 ?
   1             A           29                 ?
   2             A           31                 ?
   3             A           34                 ?
   4             A          121                 ?
   5             A          122                 ?
   6             A          123                 ?
   7             A          124                 ?
   8             A          125                 ?
   9             A          126                 ?
   10            A          127                 ?
   11            A          169                 ?
   12            A          170                 ?
   13            A          171                 ?
   14            A          172                 ?
   15            A          173                 ?
   16            A          289                 ?
   17            A        

In [35]:
import pickle

resultsf = "ALLO_results.pkl"

In [36]:
with open(resultsf, "wb") as f:
    pickle.dump(results, f)

In [37]:
with open(resultsf, "rb") as f:
    results = pickle.load(f)

results

{'4jqi': {'P_0': {'pred': 1,
   'prob': 0.023459999999999998,
   'residues':    auth_asym_id  auth_seq_id pdbx_PDB_ins_code
   0             A           26                 ?
   1             A           29                 ?
   2             A           31                 ?
   3             A           34                 ?
   4             A          121                 ?
   5             A          122                 ?
   6             A          123                 ?
   7             A          124                 ?
   8             A          125                 ?
   9             A          126                 ?
   10            A          127                 ?
   11            A          169                 ?
   12            A          170                 ?
   13            A          171                 ?
   14            A          172                 ?
   15            A          173                 ?
   16            A          289                 ?
   17            A        

In [38]:
resultsd = {
    pdb: {
        pocket: {
            k: v if k != "residues" else v.astype(str).to_dict(orient="list")
            for k, v in pocketd.items()
        }
        for pocket, pocketd in pockets.items()
    } for pdb, pockets in results.items()
}

resultsd

{'4jqi': {'P_0': {'pred': 1,
   'prob': 0.023459999999999998,
   'residues': {'auth_asym_id': ['A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A'],
    'auth_seq_id': ['26',
     '29',
     '31',
     '34',
     '121',
     '122',
     '123',
     '124',
     '125',
     '126',
     '127',
     '169',
     '170',
     '171',
     '172',
     '173',
     '289',
     '290',
     '291',
     '293',
     '296',
     '297',
     '298',
     '299',
     '301',
     '302',
     '303',
     '304',
     '305',
     '306',
     '307',
     '312',
     '314',
     '315',
     '316',
     '317',
     '318',
     '319',
     '352',
     '353',
     '36

In [39]:
import json

In [40]:
with open(resultsf.replace(".pkl", ".json"), "w") as f:
    json.dump(resultsd, f)